In [1]:
import os

import numpy as np
import pandas as pd

from ydata_profiling import ProfileReport

In [2]:
INPUT_FOLDER = "enrich"
OUTPUT_FOLDER = "aggregate"

In [3]:
os.makedirs("data/aggregate", exist_ok=True)

In [4]:
datasources = {
    source.replace(".zip",""):source
    for source in os.listdir(f"data/{INPUT_FOLDER}")
    if source.endswith(".zip")
}

datasets = {}

## AGGREGATE

### Create BASE TABLE

The base table is just a table with all the primary keys we want to maintain in the macro table. In this case we want to build a table indexed by `week` and `country_name`.
 - `week` - We will take the `epidemiology` table to get the start and end dates.
 - `country_name` - We get the `country_name` from the `index` table.
 
So, in this case, we have to build an index with all dates and all country names.

<div class="alert alert-warning">
    <b>Special mention to datetime indexes</b>: Be careful when building detetime indexes from some of the provided data tables, they may contain gaps in the timestamps. So my recommendation here is that you build the date ranges by yourself. Then, you will have time to spot missing dates and fix them
</div>

#### Get date ranges

In [5]:
key = "epidemiology"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip", parse_dates=["date"])

start_date = data.date.min()
end_date = data.date.max()
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

Start Date: 2020-01-02 00:00:00
End Date: 2022-08-22 00:00:00


In [6]:
dates = pd.date_range(start=start_date, end=end_date, freq="W")
dates = dates.to_period("W").astype(str)
dates = pd.DataFrame(dates)

#### Get country names

In [8]:
key = "index"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")

In [9]:
countries = pd.DataFrame(data.country_name.unique())

countries

,0
0,Germany
1,Spain
2,Italy
3,United States of America


#### Build the index

In [10]:
base = dates.merge(countries, how="cross")
base.columns = ["week", "country_name"]
base = base.set_index(["week", "country_name"])

In [11]:
base.head()

Empty DataFrame
Columns: []
Index: [(2019-12-30/2020-01-05, Germany), (2019-12-30/2020-01-05, Spain), (2019-12-30/2020-01-05, Italy), (2019-12-30/2020-01-05, United States of America), (2020-01-06/2020-01-12, Germany)]

In [12]:
base.shape

(552, 0)

In [13]:
macro = base.copy()

### Create MACRO TABLE

#### Incorporate `epidemiology`

In [14]:
key = "epidemiology"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161033 entries, 0 to 3161032
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   date                          3161033 non-null  object 
 1   week                          3161033 non-null  object 
 2   location_key                  3161033 non-null  object 
 3   new_confirmed                 3161033 non-null  float64
 4   new_deceased                  3161033 non-null  float64
 5   new_deceased_confirmed_ratio  2197437 non-null  float64
 6   country_name                  3161033 non-null  object 
dtypes: float64(3), object(4)
memory usage: 168.8+ MB


##### _Aggregate_: (sum) `new_confirmed`, `new_deceased`, `new_deceased_confirmed_ratio`

In [15]:
aux = data.groupby(["week","country_name"]).agg({
    "new_confirmed": "sum",
    "new_deceased": "sum",
    "new_deceased_confirmed_ratio": "mean",
})

In [16]:
macro

Empty DataFrame
Columns: []
Index: [(2019-12-30/2020-01-05, Germany), (2019-12-30/2020-01-05, Spain), (2019-12-30/2020-01-05, Italy), (2019-12-30/2020-01-05, United States of America), (2020-01-06/2020-01-12, Germany), (2020-01-06/2020-01-12, Spain), (2020-01-06/2020-01-12, Italy), (2020-01-06/2020-01-12, United States of America), (2020-01-13/2020-01-19, Germany), (2020-01-13/2020-01-19, Spain), (2020-01-13/2020-01-19, Italy), (2020-01-13/2020-01-19, United States of America), (2020-01-20/2020-01-26, Germany), (2020-01-20/2020-01-26, Spain), (2020-01-20/2020-01-26, Italy), (2020-01-20/2020-01-26, United States of America), (2020-01-27/2020-02-02, Germany), (2020-01-27/2020-02-02, Spain), (2020-01-27/2020-02-02, Italy), (2020-01-27/2020-02-02, United States of America), (2020-02-03/2020-02-09, Germany), (2020-02-03/2020-02-09, Spain), (2020-02-03/2020-02-09, Italy), (2020-02-03/2020-02-09, United States of America), (2020-02-10/2020-02-16, Germany), (2020-02-10/2020-02-16, Spain), (2020-02-10/2020-02-16, Italy), (2020-02-10/2020-02-16, United States of America), (2020-02-17/2020-02-23, Germany), (2020-02-17/2020-02-23, Spain), (2020-02-17/2020-02-23, Italy), (2020-02-17/2020-02-23, United States of America), (2020-02-24/2020-03-01, Germany), (2020-02-24/2020-03-01, Spain), (2020-02-24/2020-03-01, Italy), (2020-02-24/2020-03-01, United States of America), (2020-03-02/2020-03-08, Germany), (2020-03-02/2020-03-08, Spain), (2020-03-02/2020-03-08, Italy), (2020-03-02/2020-03-08, United States of America), (2020-03-09/2020-03-15, Germany), (2020-03-09/2020-03-15, Spain), (2020-03-09/2020-03-15, Italy), (2020-03-09/2020-03-15, United States of America), (2020-03-16/2020-03-22, Germany), (2020-03-16/2020-03-22, Spain), (2020-03-16/2020-03-22, Italy), (2020-03-16/2020-03-22, United States of America), (2020-03-23/2020-03-29, Germany), (2020-03-23/2020-03-29, Spain), (2020-03-23/2020-03-29, Italy), (2020-03-23/2020-03-29, United States of America), (2020-03-30/2020-04-05, Germany), (2020-03-30/2020-04-05, Spain), (2020-03-30/2020-04-05, Italy), (2020-03-30/2020-04-05, United States of America), (2020-04-06/2020-04-12, Germany), (2020-04-06/2020-04-12, Spain), (2020-04-06/2020-04-12, Italy), (2020-04-06/2020-04-12, United States of America), (2020-04-13/2020-04-19, Germany), (2020-04-13/2020-04-19, Spain), (2020-04-13/2020-04-19, Italy), (2020-04-13/2020-04-19, United States of America), (2020-04-20/2020-04-26, Germany), (2020-04-20/2020-04-26, Spain), (2020-04-20/2020-04-26, Italy), (2020-04-20/2020-04-26, United States of America), (2020-04-27/2020-05-03, Germany), (2020-04-27/2020-05-03, Spain), (2020-04-27/2020-05-03, Italy), (2020-04-27/2020-05-03, United States of America), (2020-05-04/2020-05-10, Germany), (2020-05-04/2020-05-10, Spain), (2020-05-04/2020-05-10, Italy), (2020-05-04/2020-05-10, United States of America), (2020-05-11/2020-05-17, Germany), (2020-05-11/2020-05-17, Spain), (2020-05-11/2020-05-17, Italy), (2020-05-11/2020-05-17, United States of America), (2020-05-18/2020-05-24, Germany), (2020-05-18/2020-05-24, Spain), (2020-05-18/2020-05-24, Italy), (2020-05-18/2020-05-24, United States of America), (2020-05-25/2020-05-31, Germany), (2020-05-25/2020-05-31, Spain), (2020-05-25/2020-05-31, Italy), (2020-05-25/2020-05-31, United States of America), (2020-06-01/2020-06-07, Germany), (2020-06-01/2020-06-07, Spain), (2020-06-01/2020-06-07, Italy), (2020-06-01/2020-06-07, United States of America), (2020-06-08/2020-06-14, Germany), (2020-06-08/2020-06-14, Spain), (2020-06-08/2020-06-14, Italy), (2020-06-08/2020-06-14, United States of America), (2020-06-15/2020-06-21, Germany), (2020-06-15/2020-06-21, Spain), (2020-06-15/2020-06-21, Italy), (2020-06-15/2020-06-21, United States of America), ...]

[552 rows x 0 columns]

In [17]:
aux

new_confirmed  new_deceased  \
week                  country_name                                            
2019-12-30/2020-01-05 Germany                             1.0           0.0   
2020-01-13/2020-01-19 Germany                             1.0           0.0   
2020-01-20/2020-01-26 Germany                             2.0           0.0   
                      United States of America            0.0           0.0   
2020-01-27/2020-02-02 Germany                            10.0           0.0   
...                                                       ...           ...   
2022-08-08/2022-08-14 Italy                          169974.0           0.0   
                      United States of America       126631.0         502.0   
2022-08-15/2022-08-21 Italy                          137729.0           0.0   
                      United States of America       110333.0         400.0   
2022-08-22/2022-08-28 Italy                            9603.0           0.0   

                                                new_deceased_confirmed_ratio  
week                  country_name                                            
2019-12-30/2020-01-05 Germany                                       0.000000  
2020-01-13/2020-01-19 Germany                                       0.000000  
2020-01-20/2020-01-26 Germany                                       0.000000  
                      United States of America                           NaN  
2020-01-27/2020-02-02 Germany                                       0.000000  
...                                                                      ...  
2022-08-08/2022-08-14 Italy                                         0.000000  
                      United States of America                      0.001763  
2022-08-15/2022-08-21 Italy                                         0.000000  
                      United States of America                      0.000879  
2022-08-22/2022-08-28 Italy                                         0.000000  

[505 rows x 3 columns]

##### _Include in the base table_

In [18]:
macro = macro.join(aux)

In [19]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
dtypes: float64(3)
memory usage: 36.2+ KB


#### `demographics`

In [20]:
key = "demographics"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5097 entries, 0 to 5096
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   location_key                 5097 non-null   object 
 1   population                   5097 non-null   float64
 2   population_age_00_09         5097 non-null   float64
 3   population_age_10_19         5097 non-null   float64
 4   population_age_20_29         5097 non-null   float64
 5   population_age_30_39         5097 non-null   float64
 6   population_age_40_49         5097 non-null   float64
 7   population_age_50_59         5097 non-null   float64
 8   population_age_60_69         5097 non-null   float64
 9   population_age_70_79         5097 non-null   float64
 10  population_age_80_and_older  5097 non-null   float64
 11  country_name                 5097 non-null   object 
dtypes: float64(10), object(2)
memory usage: 478.0+ KB


##### _Aggregate_: (sum)

In [21]:
aux = data.groupby("country_name").sum()

In [22]:
macro = macro.join(aux, on="country_name")

In [23]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
 3   location_key                  552 non-null    object 
 4   population                    552 non-null    float64
 5   population_age_00_09          552 non-null    float64
 6   population_age_10_19          552 non-null    float64
 7   population_age_20_29          552 non-null    float64
 8   population_age_30_39          552 non-null    float64
 9   population_age_40_49          552 non-null    float64
 10  population_age_50_59          552 non-null    float64
 11  population_age_60_69          552 non-

#### `health`

In [24]:
key = "health"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3022 entries, 0 to 3021
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location_key     3022 non-null   object 
 1   life_expectancy  3022 non-null   float64
 2   country_name     3022 non-null   object 
dtypes: float64(1), object(2)
memory usage: 71.0+ KB


##### _Aggregate_: (mean)

In [25]:
aux = data.groupby(["country_name"]).agg({
    "life_expectancy": "mean",
})

##### _Include in the base table_

In [26]:
macro = macro.join(aux, on="country_name")

In [27]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
 3   location_key                  552 non-null    object 
 4   population                    552 non-null    float64
 5   population_age_00_09          552 non-null    float64
 6   population_age_10_19          552 non-null    float64
 7   population_age_20_29          552 non-null    float64
 8   population_age_30_39          552 non-null    float64
 9   population_age_40_49          552 non-null    float64
 10  population_age_50_59          552 non-null    float64
 11  population_age_60_69          552 non-

#### `hospitalizations`

In [28]:
key = "hospitalizations"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6297 entries, 0 to 6296
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       6297 non-null   object 
 1   week                       6297 non-null   object 
 2   location_key               6297 non-null   object 
 3   new_hospitalized_patients  6297 non-null   float64
 4   country_name               6297 non-null   object 
dtypes: float64(1), object(4)
memory usage: 246.1+ KB


##### _Aggregate_: (sum)

In [29]:
aux = data.groupby(["week","country_name"]).sum()

In [30]:
macro = macro.join(aux, lsuffix='_macro', rsuffix='_aux')

In [31]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
 3   location_key_macro            552 non-null    object 
 4   population                    552 non-null    float64
 5   population_age_00_09          552 non-null    float64
 6   population_age_10_19          552 non-null    float64
 7   population_age_20_29          552 non-null    float64
 8   population_age_30_39          552 non-null    float64
 9   population_age_40_49          552 non-null    float64
 10  population_age_50_59          552 non-null    float64
 11  population_age_60_69          552 non-

#### `vaccinations`

In [32]:
key = "vaccinations"
data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.zip")
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562414 entries, 0 to 1562413
Data columns (total 5 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   date                          1562414 non-null  object 
 1   week                          1562414 non-null  object 
 2   location_key                  1562414 non-null  object 
 3   new_persons_fully_vaccinated  1562414 non-null  float64
 4   country_name                  1562414 non-null  object 
dtypes: float64(1), object(4)
memory usage: 59.6+ MB


##### _Aggregate_: (sum)

In [36]:
data = data.set_index("date").sort_index()
aux = data.groupby(["week", "country_name"]).agg({
    "new_persons_fully_vaccinated": sum
})

C:\Users\SLO\AppData\Local\Temp\ipykernel_21612\1624207279.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  aux = data.groupby(["week", "country_name"]).agg({


In [37]:
macro = macro.join(aux, lsuffix='_macro', rsuffix='_aux')

In [38]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
 3   location_key_macro            552 non-null    object 
 4   population                    552 non-null    float64
 5   population_age_00_09          552 non-null    float64
 6   population_age_10_19          552 non-null    float64
 7   population_age_20_29          552 non-null    float64
 8   population_age_30_39          552 non-null    float64
 9   population_age_40_49          552 non-null    float64
 10  population_age_50_59          552 non-null    float64
 11  population_age_60_69          552 non-

### Save `macrotable`

In [39]:
macro.to_csv(f"data/{OUTPUT_FOLDER}/macrotable.zip", index=True)

In [42]:
profile = ProfileReport(macro, minimal=True)
profile.to_file("profiling/macro.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\SLO\Documents\GitHub\IE-University\myenv312\Lib\site-packages\numpy\_core\_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
c:\Users\SLO\Documents\GitHub\IE-University\myenv312\Lib\site-packages\pandas\core\nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
c:\Users\SLO\Documents\GitHub\IE-University\myenv312\Lib\site-packages\pandas\core\nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
c:\Users\SLO\Documents\GitHub\IE-University\myenv312\Lib\site-packages\numpy\lib\_function_base_impl.py:4620: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Couldn't find space to draw. Either the Canvas size is too small or too much of the image is masked out.

In [43]:
macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  394 non-null    float64
 3   location_key_macro            552 non-null    object 
 4   population                    552 non-null    float64
 5   population_age_00_09          552 non-null    float64
 6   population_age_10_19          552 non-null    float64
 7   population_age_20_29          552 non-null    float64
 8   population_age_30_39          552 non-null    float64
 9   population_age_40_49          552 non-null    float64
 10  population_age_50_59          552 non-null    float64
 11  population_age_60_69          552 non-

### Final missing values review

At this moment, some missing values can arise from the resulting aggregations. This is the time to decide what to do with them to buil a clean table for EDA.

In this case:
 - `new_confirmed` and `new_deceased`: These missings are because some countries doesn't have the starting date at the same time as the country with the earliest record.
    - **Action: Remove all missing rows**
 - `new_deceased_confirmed_ratio`: This value doesn't have missing values, but infinite values due to dividing by 0.
    - **Action: Impute the value as 0**
 - `life_expectancy`: These missings are because this variable just applies to the United States. For those missings related to the United States, impute values to the mean
    - **Action: Impute values to the mean, but just for the United States**
 - `new_hospitalized_patients`: These missings are because this variable just applies to the United States. For those missings related to the United States, impute values to the mean
    - **Action: Impute values to the mean, but just for the United States**
 - `new_persons_fully_vaccinated`: These missings are because this variable just applies to the United States. For those missings related to the United States, impute values to 0 (there was no vaccination.) 
    - **Action: Impute values to 0, but just for the United States**

In [44]:
clean_macro = macro.copy()

Imputing `new_confirmed`

In [45]:
clean_macro = clean_macro.dropna(subset=["new_confirmed"])

Imputing `new_deceased_confirmed_ratio`

In [46]:
clean_macro = clean_macro.fillna({
    "new_deceased_confirmed_ratio": 0
})

Imputing `life_expectancy` and `new_hospitalized_patients`

In [47]:
us_missings = clean_macro.query(
    "country_name == 'United States of America'"
)[["life_expectancy","new_hospitalized_patients"]]
us_missings.head()

,,life_expectancy,new_hospitalized_patients
week,country_name,,
2020-01-20/2020-01-26,United States of America,77.871999,NaN
2020-01-27/2020-02-02,United States of America,77.871999,NaN
2020-02-03/2020-02-09,United States of America,77.871999,NaN
2020-02-10/2020-02-16,United States of America,77.871999,NaN
2020-02-17/2020-02-23,United States of America,77.871999,NaN


In [48]:
us_missings = us_missings.groupby(level="country_name").mean()
us_missings

,life_expectancy,new_hospitalized_patients
country_name,,
United States of America,77.871999,2544.6


In [49]:
clean_macro = clean_macro.fillna(us_missings)

Imputing `new_persons_fully_vaccinated`

In [50]:
clean_macro.loc[
    (clean_macro.index.get_level_values("country_name")=="United States of America")
    & (clean_macro.new_persons_fully_vaccinated.isna()),
    "new_persons_fully_vaccinated"
] = 0

In [51]:
clean_macro.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 504 entries, ('2019-12-30/2020-01-05', 'Germany') to ('2022-08-15/2022-08-21', 'United States of America')
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 504 non-null    float64
 1   new_deceased                  504 non-null    float64
 2   new_deceased_confirmed_ratio  504 non-null    float64
 3   location_key_macro            504 non-null    object 
 4   population                    504 non-null    float64
 5   population_age_00_09          504 non-null    float64
 6   population_age_10_19          504 non-null    float64
 7   population_age_20_29          504 non-null    float64
 8   population_age_30_39          504 non-null    float64
 9   population_age_40_49          504 non-null    float64
 10  population_age_50_59          504 non-null    float64
 11  population_age_60_69          504 non-

### Save cleaned macrotable

In [52]:
clean_macro.to_csv(f"data/{OUTPUT_FOLDER}/macrotable_c.zip", index=True)